# Chapter 15: Introduction to Objects and classes.

I will only briefly dive into classes here, there is lots more to know.

These two chapters are more a "get your feet wet" tour of classes, so that you'll have an idea what to do when you find classes used by someone else's code, rather than preparing you to design large class-based software projects

In [ ]:
#I'll start with an example of a trivial class so we can see some syntax.
class SimpleClass:
    """A string enclosed in triple quotes is often used to provide documentation
    for the class. Note that the name is capitalized on the first letter.
    Following the documentation, I provide names of members for the class."""
    a = 1234
    def f(self):
        return "Hello, Classes!"

#We'll get to that "self" business in a second, promise.
#For now, let's play with our class.
    
def useSimpleClass():
    #Make an instance of simpleClass, assign it to the name simpleObject.
    simpleObject = SimpleClass() #I discuss what this does below, don't fret.
    print(simpleObject.a)
    print(simpleObject.f())

    #Some subtlety of assignment:
    sc1 = SimpleClass() #sc1 is an object on the heap. 
    sc2 = sc1             #sc2 is the same as sc1 - it is a pointer to sc1.
    sc3 = SimpleClass() #sc3 is an object on the heap, separate from sc1.
    def printEm(x,y,z):
        print("a.a = {0}, b.a = {1}, c.a = {2}".format(x.a,y.a,z.a))
    printEm(sc1,sc2,sc3)
    sc1.a = 5
    printEm(sc1,sc2,sc3)
    sc3.a = 2
    printEm(sc1,sc2,sc3)
    sc2.a = 3
    printEm(sc1,sc2,sc3)

We good so far? Let me formalize some terminology:

A *class* is a specification, a blueprint.

*SimpleClass* is a class that specifies something that contains the number 1234 and a function that returns a string.

When I instantiate the class, I call it like a function with no parameters.

This creates an *object*, which is a physical manifestation of that class.

Calling *SimpleClass*() creates some space on the heap and stores the object there, and does some other subtle things needed to get the object ready to use.

A *member* is a variable stored in an object. In *SimpleClass*, *a* is a member.

A *method* is a function stored in an object. In *SimpleClass*, *f* is a method. 

Following this are a bunch of different things we can do with a class. I'll build on *SimpleClass* throughout this chapter, but it should be possible to skip one segment as they are pretty separate.

First, let me deal with that "self" business. Think of it like this: Whenever you call a method on an object, it is rewritten:

In [ ]:
#objectName.methodName() -> methodName(objectName)

If the method has arguments, it becomes

In [ ]:
#objectName.methodName(a,b,c) -> methodName(objectName,a,b,c)

This is very powerful, it allows us to inspect our current object. If you followed Chapter 13 all the way through (congratulations!) you saw a funky way to mutate a hidden variable. With classes, this is normal.

In [ ]:
class SC1:
    """Here, I have added a getter and setter for some attributes."""
    a = 1234
    b = "Hello, Getters."
    def getA(self):
        return self.a
    
    def setA(self, newVal):
        print("a was just changed from {0:d} to {1:d}".format(self.a, newVal))
        self.a = newVal

    def getB(self):
        return self.b

    def f(self):
        return "Hello, Classes!"

A note on politeness. It is generally considered rude to access any members (or methods, for that matter) that start with an underscore. 

In *SC1.setA*, I log each time someone changes the value of *a*. But, someone could easily foil this by assigning to *a* directly:

In [ ]:
def useSC1():
    sC = SC1()
    sc.a = 2
    sc.setA(3)
    #prints that the value was changed from 2 to 3, but never mentions that it
    #was changed to 2 in the first place. This is a great way to drive
    #a maintenance programmer bonkers.

class SC2:
    _a = 1234
    _b = "Hello, protection."
    def getA(self):
        return self._a
    
    def setA(self, newVal):
        print("a was just changed from {0:d} to {1:d}".format(self._a, newVal))
        self._a = newVal

    def getB(self):
        return self._b

    #This method should not be accessed by outside code. 
    def _setB(self, newVal):
        self._b = newVal
        
    def changeB(self, newVal):
        if(len(newVal) < 50):
            self._setB(newVal)
        else:
            raise OverflowError

    def f(self):
        return "Hello, Classes!"




Okay, next:
We instantiate with *ClassName*().

As you might imagine, we can put things in that argument list. 

In [ ]:
#We define a function __init__(...) that does setup.

In [ ]:
class SC3:
    #Note the lack of any members. I create them in __init__.

    def __init__(self, aVal):
        self._a = aVal #_a didn't exist before, but now that I've assigned to it
                       #it is a part of this object. 
        self._b = "Hello, initializers."
    def getA(self):
        return self._a
    
    def setA(self, newVal):
        print("a was just changed from {0:d} to {1:d}".format(self._a, newVal))
        self._a = newVal

    def getB(self):
        return self._b

    def _setB(self, newVal):
        self._b = newVal
        
    def changeB(self, newVal):
        if(len(newVal) < 50):
            self._setB(newVal)

    def f(self):
        return "Hello, Classes!"


def useSC3():
    sc = SC3(4321)
    print(sc.getB())
    sc.setA(3)
    print(sc.getA())



In [ ]:
#Okay, so that "assign to a member to create it" behavior in that initializer is not limited to __init__().

#In fact, I can do it *anywhere*. Look, here I'll punch in a new field into a simple class from *somewhere totally different*:

class Stupid:
    a = 5

def playStupid():
    s = Stupid()
    s.b=15
    print(s.a+s.b)
    #You can, in theory, punch in methods, but the "self as the first argument"
    #thing gets really messy, so let's not do that.

This technique is called "duck-punching" or "monkey-patching".

The reason I introduce this is that you're going to encounter bugs, and it'll be because you're inadvertently duck-punching:

In [ ]:
class NumHolder:
    _a=5
    def getA(self):
        return self._a
    def setA(self, newA):
        self._A = newA

def playOops():
    o = NumHolder()
    print(o.getA())
    o.setA(50)
    print(o.getA())


# Drat. Okay, why was _a not updated?

# It's because I accidentally capitalized _a to _A in setA. Unfortunately, this sort of bug is *very* hard to find,
# and you're *going* to make it. At least now you are aware of it, and know to look for it.

####   *Exercises*

1 -  Create a class that represents a playing card. Its constructor should take two arguments: *rank* and *suit*.

In [1]:
#It should have at least one other method, called __str__(self), that returns a string saying what the card is.
#Print will automatically try to call __str__ when it tries to print your object.

#Example:
#c = Card("ace", "diamonds")
#print(c) #should print "ace of diamonds".
class Card:
    pass

2 - Write a function that takes two cards and returns 1 if the first one is higher-valued, 0 if they are identical, and -1 if the second is higher-valued.

As a reminder, value is determined first by rank (all fives outrank all threes) and then by suit (spades outrank diamonds, which outrank hearts, which outrank clubs).